**Table of contents**<a id='toc0_'></a>    
- 1. [Loading the Required Packages](#toc1_)    
- 2. [Geospatial Gridding](#toc2_)    
- 3. [Reprojection](#toc3_)    
- 4. [Clipping Rasters given a shapefile](#toc4_)    
- 5. [Extracting Environmental Variables](#toc5_)    
  - 5.1. [Based on Africa 5KM Grids](#toc5_1_)    
    - 5.1.1. [Temperature](#toc5_1_1_)    
    - 5.1.2. [Relative Humidity](#toc5_1_2_)    
    - 5.1.3. [Precipitation](#toc5_1_3_)    
    - 5.1.4. [NDVI](#toc5_1_4_)    
    - 5.1.5. [Build-Up](#toc5_1_5_)    
    - 5.1.6. [Irrigation](#toc5_1_6_)    
    - 5.1.7. [LULC](#toc5_1_7_)    
    - 5.1.8. [Elevation](#toc5_1_8_)    
  - 5.2. [Based on Species Data](#toc5_2_)    
    - 5.2.1. [Loading the species occurrence data.](#toc5_2_1_)    
    - 5.2.2. [Temperature](#toc5_2_2_)    
    - 5.2.3. [Relative Humidity](#toc5_2_3_)    
    - 5.2.4. [Precipitation](#toc5_2_4_)    
    - 5.2.5. [NDVI](#toc5_2_5_)    
    - 5.2.6. [Build-Up](#toc5_2_6_)    
    - 5.2.7. [Irrigation](#toc5_2_7_)    
    - 5.2.8. [LULC](#toc5_2_8_)    
    - 5.2.9. [Elevation](#toc5_2_9_)    
    - 5.2.10. [Shrubs](#toc5_2_10_)    
    - 5.2.11. [Climate Moisture Index](#toc5_2_11_)    
    - 5.2.12. [Cloud Area Fraction](#toc5_2_12_)    
    - 5.2.13. [Wind Speed](#toc5_2_13_)    
    - 5.2.14. [Potential Evapotranspiration](#toc5_2_14_)    
    - 5.2.15. [Solar Radiation](#toc5_2_15_)    
    - 5.2.16. [Exporting the data](#toc5_2_16_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[Loading the Required Packages](#toc0_)

In [1]:
import os
import csv
import glob
import shutil
import rasterio
import statistics
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rx
from shapely import wkt
import geopandas as gpd
import multiprocessing as mp
from multiprocessing import Pool
from shapely.geometry import box
from shapely.geometry import mapping

print(os.getcwd()) # Print current working directory.

d:\Kanji_Projects\ICIPE_2023\Eric\codes2\codes2


# 2. <a id='toc2_'></a>[Geospatial Gridding](#toc0_)

In [ ]:
# Creating Geospatial Grids with Numpy, Shapely & Geopandas with a geographic CRS.
# Load the shapefile.
shapefile_path = '/path/Africa_shapefile.shp'
shp = gpd.read_file(shapefile_path)

# Calculate bounding box of the shapefile
xmin, ymin, xmax, ymax = shp.total_bounds

# Define cell/pixel resolution in degrees
distance = 5 # In Kilometers
resolution = distance / 111.32  # 1 degree = 111.32km
cell_size = resolution  # Adjust as needed

# Generate grid
x = np.arange(xmin, xmax, cell_size)
y = np.arange(ymin, ymax, cell_size)
xx, yy = np.meshgrid(x, y)

# Convert grid to GeoDataFrame
polygons = [box(x, y, x + cell_size, y + cell_size) for x, y in zip(xx.ravel(), yy.ravel())]
Africa_5KM_Grid_coord = gpd.GeoDataFrame(geometry=polygons, crs=shp.crs)

# Get the grid cells that intersect the shapefile.
Africa_5KM_Grid_coord2 = gpd.sjoin(left_df=Africa_5KM_Grid_coord, right_df=shp, how="inner")

# Plot the shapefile with the grids overlay
ax = shp.plot(alpha=0.5)
Africa_5KM_Grid_coord.plot(ax=ax, color="red", alpha=0.5)
Africa_5KM_Grid_coord2.plot(ax=ax, color="green", alpha=0.5)

Africa_5KM_Grid_coord2.to_csv(r'Africa_5KM_Grid_coord.csv')

# 3. <a id='toc3_'></a>[Reprojection](#toc0_)

In [ ]:
# Automating the crs reprojection of multiple rasters to a target shapefile or vector crs.
# This is done for every raster with projected crs.

# Input and output folders
input_folder = r"E:\path\dir"
output_folder = r"E:\path\dir_projected_crs"

# Loading the  Shapefile
shapefile_path = r"C:\path\shapefile.shp"
shapefile = gpd.read_file(shapefile_path)

# Get a list of raster input files
input_files = glob.glob(os.path.join(input_folder, "*.tif"))

# Process each input file
for input_file in input_files:
    # Get the file name and extension
    file_name = os.path.basename(input_file)
    file_name_no_ext, file_ext = os.path.splitext(file_name)

    # Loading the raster
    raster = rxr.open_rasterio(input_file, masked=True).squeeze()

    # Write the CRS of the shapefile to the raster file
    raster.rio.write_crs(shapefile.crs, inplace=True)

    # Output file path
    output_file = os.path.join(output_folder, file_name)

    # Export the clipped raster as a tif file
    raster.rio.to_raster(output_file)

    # Deleting the processed and exported raster to free up memory
    del raster

    print(f"Processed: {file_name}")

print("All files processed.")

# 4. <a id='toc4_'></a>[Clipping Rasters given a shapefile](#toc0_)

In [ ]:
# Automating the clipping of multiple rasters using a single shapefile or vector
# This is done for every raster that covers more than our study area (Africa)

# Input and output raster folders folders
input_folder = r"F:\path\dir" # Unclipped raster folder.
output_folder = r"F:\path\dir_ouput" # Clipped raster folder.

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loading a Shapefile
Africa_shp = gpd.read_file(r"F:\path\Africa_shapefile.shp") # Read Africa Shapefile.

# Get a list of input files
input_files = glob.glob(os.path.join(input_folder, "*.tif"))

# Process each input file
for input_file in input_files:
    # Get the file name and extension
    file_name = os.path.basename(input_file)
    file_name_no_ext, file_ext = os.path.splitext(file_name)

    # Loading the raster
    raster = rxr.open_rasterio(input_file, masked=True).squeeze()

    # Write the CRS of the shapefile to the raster file
    raster.rio.write_crs(shp.crs, inplace=True)

    # Clipping/Cropping the raster according to the shapefile
    clipped_raster = raster.rio.clip(shp.geometry.apply(mapping), shp.crs)

    # Output file path
    output_file = os.path.join(output_folder, file_name)

    # Export the clipped raster as a tif file
    clipped_raster.rio.to_raster(output_file)

    print(f"Processed: {file_name}")

print("All files processed.")

# 5. <a id='toc5_'></a>[Extracting Environmental Variables](#toc0_)

## 5.1. <a id='toc5_1_'></a>[Based on Africa 5KM Grids](#toc0_)

### 5.1.1. <a id='toc5_1_1_'></a>[Temperature](#toc0_)

In [ ]:
input_dir = r"E:\path\dir" # Path to where downloaded rasters are stored.
output_dir =  r"E:\path\dir_output" # Path to where extracted data is stored.
csv_file = r"E:\path\Africa_5KM_Grid_coord.csv" # Path to where centroids of the grids are stored.
error_dir = r"E:\path\dir_errors" # Path to dir that store any raster with errors.

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        print(f'Accessing {filename}')
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        error_path = os.path.join(error_dir, filename)

        try:
            # open input geotiff raster
            with rasterio.open(input_path, driver='GTiff') as src:
                # extract pixel values for each coordinate = 
                values = [x[0] for x in src.sample(coordinates)]
            # write extracted data to output CSV file
            with open(output_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['lot', 'lat', 'pixel_id', 'tmax'])
                for i, (lon, lat, pixel_id) in enumerate(coordinates):
                    writer.writerow([lon, lat, pixel_id, values[i]])

            print(f'Finished extracting {filename}')

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            # Copy the file to the error directory
            shutil.copy(input_path, error_path)
            print(f"File {filename} moved to {error_dir}")
            continue  # Skip to the next file if there is an error.              

### 5.1.2. <a id='toc5_1_2_'></a>[Relative Humidity](#toc0_)

In [ ]:
input_dir = r"E:\path\dir" # Path to where downloaded rasters are stored.
output_dir =  r"E:\path\dir_output" # Path to where extracted data is stored.
csv_file = r"E:\path\Africa_5KM_Grid_coord.csv" # Path to where centroids of the grids are stored.
error_dir = r"E:\path\dir_errors" # Path to dir that store any raster with errors.

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        print(f'Accessing {filename}')
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        error_path = os.path.join(error_dir, filename)

        try:
            # open input geotiff raster
            with rasterio.open(input_path, driver='GTiff') as src:
                # extract pixel values for each coordinate = 
                values = [x[0] for x in src.sample(coordinates)]
            # write extracted data to output CSV file
            with open(output_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['lot', 'lat', 'pixel_id', 'rh'])
                for i, (lon, lat, pixel_id) in enumerate(coordinates):
                    writer.writerow([lon, lat, pixel_id, values[i]])

            print(f'Finished extracting {filename}')

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            # Copy the file to the error directory
            shutil.copy(input_path, error_path)
            print(f"File {filename} moved to {error_dir}")
            continue  # Skip to the next file if there is an error.              

### 5.1.3. <a id='toc5_1_3_'></a>[Precipitation](#toc0_)

In [ ]:
input_dir = r"E:\path\dir" # Path to where downloaded rasters are stored.
output_dir =  r"E:\path\dir_output" # Path to where extracted data is stored.
csv_file = r"E:\path\Africa_5KM_Grid_coord.csv" # Path to where centroids of the grids are stored.
error_dir = r"E:\path\dir_errors" # Path to dir that store any raster with errors.

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        print(f'Accessing {filename}')
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        error_path = os.path.join(error_dir, filename)

        try:
            # open input geotiff raster
            with rasterio.open(input_path, driver='GTiff') as src:
                # extract pixel values for each coordinate = 
                values = [x[0] for x in src.sample(coordinates)]
            # write extracted data to output CSV file
            with open(output_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['lot', 'lat', 'pixel_id', 'prec'])
                for i, (lon, lat, pixel_id) in enumerate(coordinates):
                    writer.writerow([lon, lat, pixel_id, values[i]])

            print(f'Finished extracting {filename}')

        except Exception as e:
            print(f"Error processing {filename}: {e}")
            # Copy the file to the error directory
            shutil.copy(input_path, error_path)
            print(f"File {filename} moved to {error_dir}")
            continue  # Skip to the next file if there is an error.              

### 5.1.4. <a id='toc5_1_4_'></a>[NDVI](#toc0_)

In [ ]:
input_dir = r"E:\path\NDVI_dir"
output_dir =  r"E:\path\output_dir"
africa_grid_coord = pd.read_csv('selected_countries_coord.csv')
coord_pair_list = list(zip(africa_grid_coord.lon, africa_grid_coord.lat, africa_grid_coord.Index_ID))

# loop over each input tile
for filename in os.listdir(input_dir):
    try:
        if filename.endswith('.tif'):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))

            #Read the raster 
            ndvi = rx.open_rasterio(input_path)
            ndvi_dt = ndvi.to_dataset(name='ndvi')

            start_date = '2000-02-18'
            dates = pd.date_range(start=start_date, periods=len(ndvi_dt['band']), freq='16D')

            time_da = xr.DataArray(dates, dims=('band',), name='time')
            ndvi_dt2 = ndvi_dt.assign(band=time_da)

            def extract_point_data(coord_pair):
                lon, lat, ix = coord_pair
                data = ndvi_dt2.sel(x=lon, y=lat, method='nearest')
                data_df = data.to_dataframe()
                data_df['Index_ID'] = ix
                data_df.to_csv(output_path, mode='a')

            for coord_pair in coord_pair_list:
                results = extract_point_data(coord_pair)
        
    except Exception as e:
        print("An error occurred:", str(e))
        continue  # Continue to the next iteration of the loop        

### 5.1.5. <a id='toc5_1_5_'></a>[Build-Up](#toc0_)

In [ ]:
input_dir = r"E:\path\dir_input"
output_dir =  r"E:\path\dir_output"
csv_file = r"E:\path\Africa_Grid_coord.csv"

# Extracting point data from the multiple rasters

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        # open input geotiff raster
        with rasterio.open(input_path) as src:
            # extract pixel values for each coordinate
            values = [x[0] for x in src.sample(coordinates)]
        # write extracted data to output CSV file
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Longitude', 'Latitude', 'Index_ID', 'Build_UP'])
            for i, (lon, lat, pixel_id) in enumerate(coordinates):
                writer.writerow([lon, lat, pixel_id, values[i]])
                
print('Data Extraction Complete')  

### 5.1.6. <a id='toc5_1_6_'></a>[Irrigation](#toc0_)

In [ ]:
input_dir = r"E:\path\dir_input"
output_dir =  r"E:\path\dir_output"
csv_file = r"E:\path\Africa_Grid_coord.csv"

# Extracting point data from the multiple rasters

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        # open input geotiff raster
        with rasterio.open(input_path) as src:
            # extract pixel values for each coordinate
            values = [x[0] for x in src.sample(coordinates)]
        # write extracted data to output CSV file
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Longitude', 'Latitude', 'Index_ID', 'Irrigation'])
            for i, (lon, lat, pixel_id) in enumerate(coordinates):
                writer.writerow([lon, lat, pixel_id, values[i]])
                
print('Data Extraction Complete')  

### 5.1.7. <a id='toc5_1_7_'></a>[LULC](#toc0_)

In [ ]:
input_dir = r"E:\path\dir_input"
output_dir =  r"E:\path\dir_output"
csv_file = r"E:\path\Africa_Grid_coord.csv"

# Extracting point data from the multiple rasters

# read coordinates from CSV file, skipping the header row
with open(csv_file, 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    coordinates = [(float(row[0]), float(row[1]), int(row[2])) for row in reader]

# loop over each input tile.
for filename in os.listdir(input_dir):
    if filename.endswith('.tif'):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename.replace('.tif', '.csv'))
        # open input geotiff raster
        with rasterio.open(input_path) as src:
            # extract pixel values for each coordinate
            values = [x[0] for x in src.sample(coordinates)]
        # write extracted data to output CSV file
        with open(output_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Longitude', 'Latitude', 'Index_ID', 'LULC'])
            for i, (lon, lat, pixel_id) in enumerate(coordinates):
                writer.writerow([lon, lat, pixel_id, values[i]])
                
print('Data Extraction Complete')  

### 5.1.8. <a id='toc5_1_8_'></a>[Elevation](#toc0_)

In [ ]:
%%time

coord_pair_list = list(zip(Africa_coord.lon, Africa_coord.lat, Africa_coord.Index_ID))
# print(coord_pair_list)

elevation =  rx.open_rasterio(r"E:\path\elevation.tif")
# elevation

elevation_dt = elevation.to_dataset(name='elevation')

def extract_elevation_data(coord_pair):
    lon, lat, ix = coord_pair
    data = elevation_dt.sel(x=lon, y=lat, method='nearest')
    data_df = data.to_dataframe()
    data_df['Index_ID'] = ix
    data_df.to_csv(r"E:\path\elevation.csv", mode='a')
    return data_df


for coord_pair in coord_pair_list:
    results = extract_elevation_data(coord_pair)
# print(results)

print('Finished Extracting Data Successfully')

## 5.2. <a id='toc5_2_'></a>[Based on Species Data](#toc0_)

### 5.2.1. <a id='toc5_2_1_'></a>[Loading the species occurrence data.](#toc0_)

In [ ]:
coord_data = pd.read_csv(r'path_to_species_occurrence_data')
coord_data

### 5.2.2. <a id='toc5_2_2_'></a>[Temperature](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_temp_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_temp_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "Temp" not in coord_data.columns:
        coord_data["Temp"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "Temp"] = mean_value

### 5.2.3. <a id='toc5_2_3_'></a>[Relative Humidity](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_rh_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_rh_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "RH" not in coord_data.columns:
        coord_data["RH"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "RH"] = mean_value

### 5.2.4. <a id='toc5_2_4_'></a>[Precipitation](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_prec_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_prec_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "Prec" not in coord_data.columns:
        coord_data["Prec"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "Prec"] = sum_value

### 5.2.5. <a id='toc5_2_5_'></a>[NDVI](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"D:\Kanji_Projects\Data\Vegetation\NDVI\Unstacked_Mosaic"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if year_value < 2000:
            file_name = f"NDVI_{2000}-0{2}-01.tif"

        elif year_value == 2000 and month_value <= 2:
            file_name = f"NDVI_{2000}-0{2}-01.tif"

        elif year_value >= 2000 and month_value <= 9:
            file_name = f"NDVI_{year_value}-0{month_value}-01.tif"

        else:
            file_name = f"NDVI_{year_value}-{month_value}-01.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "NDVI" not in coord_data.columns:
        coord_data["NDVI"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "NDVI"] = mean_value

### 5.2.6. <a id='toc5_2_6_'></a>[Build-Up](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"E:\path\dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        # month_value = date.month
        year_value = date.year

        if year_value < 2001:
            file_name = f"built_up_{2001}.tif"

        else:
            file_name = f"built_up_{year_value}.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    # mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)
    mode_value = statistics.mode(data_list)

    # Check if the column exists, and create it if not
    if "Built_Up" not in coord_data.columns:
        coord_data["Built_Up"] = pd.Series(dtype=int)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "Built_Up"] = mode_value

### 5.2.7. <a id='toc5_2_7_'></a>[Irrigation](#toc0_)

In [ ]:
# Input folder
input_path = r"E:\path\raster.tif"

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    lon = row["longitude"]
    lat = row["latitude"]

    # Loading the raster data
    raster_data = rx.open_rasterio(input_path)

    # Extract point data.
    point_data = raster_data.sel(x=lon, y=lat, method="nearest")

    # Extract the value(s) from the point data.
    value = point_data.item()

    # Check if the column exists, and create it if not
    if "Irrigation" not in coord_data.columns:
        coord_data["Irrigation"] = pd.Series(dtype=int)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "Irrigation"] = value

### 5.2.8. <a id='toc5_2_8_'></a>[LULC](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"E:\path\dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        # month_value = date.month
        year_value = date.year

        if year_value < 2001:
            file_name = f"MODIS_lulc_img_{2001}_01_01_LC_Type1_africa.tif"

        else:
            file_name = f"MODIS_lulc_img_{year_value}_01_01_LC_Type1_africa.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    # mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)
    mode_value = statistics.mode(data_list)

    # Check if the column exists, and create it if not
    if "LULC" not in coord_data.columns:
        coord_data["LULC"] = pd.Series(dtype=int)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "LULC"] = mode_value

### 5.2.9. <a id='toc5_2_9_'></a>[Elevation](#toc0_)

In [ ]:
# Input folder
input_path = r"E:\path\raster.tif"

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    lon = row["longitude"]
    lat = row["latitude"]

    # Loading the raster data
    raster_data = rx.open_rasterio(input_path)

    # Extract point data.
    point_data = raster_data.sel(x=lon, y=lat, method="nearest")

    # Extract the value(s) from the point data.
    value = point_data.item()

    # Check if the column exists, and create it if not
    if "Elevation" not in coord_data.columns:
        coord_data["Elevation"] = pd.Series(dtype=int)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "Elevation"] = value

### 5.2.10. <a id='toc5_2_10_'></a>[Shrubs](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"E:\path\dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        # month_value = date.month
        year_value = date.year

        if year_value < 2001:
            file_name = f"shrubs_{2001}.tif"

        else:
            file_name = f"shrubs_{year_value}.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    # mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)
    mode_value = statistics.mode(data_list)

    # Check if the column exists, and create it if not
    if "shrubs" not in coord_data.columns:
        coord_data["shrubs"] = pd.Series(dtype=int)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "shrubs"] = mode_value

### 5.2.11. <a id='toc5_2_11_'></a>[Climate Moisture Index](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_cmi_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_cmi_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "CMI" not in coord_data.columns:
        coord_data["CMI"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "CMI"] = mean_value

### 5.2.12. <a id='toc5_2_12_'></a>[Cloud Area Fraction](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_caf_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_caf_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "CAF" not in coord_data.columns:
        coord_data["CAF"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "CAF"] = mean_value

### 5.2.13. <a id='toc5_2_13_'></a>[Wind Speed](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_ws_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_ws_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "WS" not in coord_data.columns:
        coord_data["WS"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "WS"] = mean_value

### 5.2.14. <a id='toc5_2_14_'></a>[Potential Evapotranspiration](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_pe_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_pe_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "PE" not in coord_data.columns:
        coord_data["PE"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "PE"] = mean_value

### 5.2.15. <a id='toc5_2_15_'></a>[Solar Radiation](#toc0_)

In [ ]:
# Input folder
input_dir_path = r"path_to_dir"
# start_index = 28428

# Iterate through the specified columns
# for index, row in coord_data.loc[start_index:].iterrows():
for index, row in coord_data.iterrows():
    print(f"Accessing row in Index {index}")
    start_date = pd.to_datetime(f"{row['month_st']}-{row['year_st']}", format="%m-%Y")
    end_date = pd.to_datetime(f"{row['month_end']}-{row['year_end']}", format="%m-%Y")
    lon = row["longitude"]
    lat = row["latitude"]

    # Generate date range and append to the list
    date_range = pd.date_range(start=start_date, end=end_date, freq="M")
    date_range = date_range.union([end_date])
    # print(date_range)

    # Create an empty list.
    data_list = []

    # Iterate through each date in the date range
    for date in date_range:
        # Access the month and year values
        month_value = date.month
        year_value = date.year

        if month_value <= 9:
            file_name = f"CHELSA_sr_0{month_value}_{year_value}_V.2.1.tif"
        else:
            file_name = f"CHELSA_sr_{month_value}_{year_value}_V.2.1.tif"

        # Input file path
        input_file = os.path.join(input_dir_path, file_name)

        # Loading the raster data
        raster_data = rx.open_rasterio(input_file)

        # Extract point data.
        point_data = raster_data.sel(x=lon, y=lat, method="nearest")

        # Extract the value(s) from the point data.
        value = point_data.item()

        data_list.append(value)

    mean_value = statistics.mean(data_list)
    # sum_value = sum(data_list)

    # Check if the column exists, and create it if not
    if "SR" not in coord_data.columns:
        coord_data["SR"] = pd.Series(dtype=float)

    # Update the DataFrame with the mean value
    coord_data.loc[index, "SR"] = mean_value

### 5.2.16. <a id='toc5_2_16_'></a>[Exporting the data](#toc0_)

In [ ]:
coord_data.to_csv('path')